In [1]:
import pandas as pd
import numpy

In [2]:
covid_dataset = pd.read_csv('dpc-covid19-ita-regioni.csv')

In [6]:
covid_dataset

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17929,2022-06-26T17:00:00,ITA,19,Sicilia,38.115697,13.362357,703,23,726,65749,...,DECEDUTI: N. 5 IL 25/06/2022 - N. 4 IL 24/06/2...,2.0,NaN,Sul numero complessivo dei casi confermati com...,510889.0,764028.0,4795562.0,8715227.0,ITG,ITG1
17930,2022-06-26T17:00:00,ITA,9,Toscana,43.769231,11.255889,337,12,349,42163,...,NaN,2.0,NaN,NaN,642555.0,553100.0,6831853.0,7578364.0,ITI,ITI1
17931,2022-06-26T17:00:00,ITA,10,Umbria,43.106758,12.388247,128,2,130,11061,...,- Si fa presente che 4 dei ricoveri non UTI ap...,0.0,NaN,NaN,142855.0,157820.0,1661691.0,2719020.0,ITI,ITI2
17932,2022-06-26T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,9,0,9,627,...,NaN,0.0,NaN,NaN,12544.0,25041.0,140666.0,384873.0,ITC,ITC2


In [3]:
# This function merge the data of Trento and Bolzano in one region
def merge_trentino(data_covid):
    
    data_covid = data_covid.drop(['stato','codice_regione','lat','long','variazione_totale_positivi','casi_da_sospetto_diagnostico','casi_da_screening','casi_testati','note','ingressi_terapia_intensiva','note_test','note_casi','totale_positivi_test_molecolare','totale_positivi_test_antigenico_rapido','tamponi_test_molecolare','tamponi_test_antigenico_rapido','codice_nuts_1','codice_nuts_2','nuovi_positivi'], axis=1)
    data_covid['data'] = data_covid['data'].map(lambda x: x[:10])
    #Salvo index 
    num_regione = len(data_covid[data_covid['denominazione_regione'] == 'Abruzzo'])
    index = list(range(0, data_covid.index[-1] - num_regione +1))
    
    #Salvo date
    list_dates = list(set(data_covid['data']))
    list_dates.sort()
    
    clean = data_covid.drop(data_covid[data_covid.denominazione_regione == 'P.A. Trento'].index | data_covid[data_covid.denominazione_regione == 'P.A. Bolzano'].index)

    
    data_covid = data_covid.loc[(data_covid['denominazione_regione'] == 'P.A. Bolzano') | (data_covid['denominazione_regione'] == 'P.A. Trento')]
    group = data_covid.groupby(by=["data"]).sum()
    
    group.insert(0,'data',list_dates)
    group.insert(1,'denominazione_regione','Trentino-Alto Adige')
    
    
    result = pd.concat([clean, group])
    
    result = result.sort_values(by=['data', 'denominazione_regione'])
    
    result.index = index
    
    return result

In [7]:
# This function compute the increment for each metric in each day.
def order_columns_right(data):
    columns = data.columns[2:]
    for column in columns:
        array = data[column].to_numpy()
        new_column = []
        
        current = 0
        for elem in array:
            if (elem - current > 0):
                incr = elem - current
                new_column.append(incr)
            else:
                new_column.append(0)
            current = elem    
        data[column] = new_column
    return data

In [8]:
# The function apply the preprocessing for each region
def manipulate_data(data):
    regions = list(set(data['denominazione_regione']))
    empty = pd.DataFrame()
    for region in regions:
        data_region = data[data['denominazione_regione'] == region]
        data_region_ordered = order_columns_right(data_region)
        empty = pd.concat([data_region_ordered,empty])
    empty = empty.sort_values(by=['data', 'denominazione_regione'])
    return empty

In [9]:
merged_dataset = merge_trentino(covid_dataset)

/var/folders/v3/v_c3jm750996kx0ctfz5pkk00000gn/T/ipykernel_3176/1881094879.py:14: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  clean = data_covid.drop(data_covid[data_covid.denominazione_regione == 'P.A. Trento'].index | data_covid[data_covid.denominazione_regione == 'P.A. Bolzano'].index)


In [10]:
final_dataset = manipulate_data(merged_dataset)

/var/folders/v3/v_c3jm750996kx0ctfz5pkk00000gn/T/ipykernel_3176/2227997666.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column] = new_column


In [11]:
# I rename in english all the attributes

final_dataset.rename(columns = {'ricoverati_con_sintomi':'hospitalized_with_symptoms', 'terapia_intensiva':'intensive_care', 'totale_ospedalizzati':'total_hospitalized', 'isolamento_domiciliare':'home_isolation', 'totale_positivi':'total_positives', 'dimessi_guariti':'discharged_healed', 'deceduti':'deceased', 'totale_casi':'total_cases', 'tamponi':'swabs'}, inplace = True)

In [13]:
# I write the result in a csv file

final_dataset.to_csv('final_dataset.csv')

In [12]:
final_dataset

,data,denominazione_regione,hospitalized_with_symptoms,intensive_care,total_hospitalized,home_isolation,total_positives,discharged_healed,deceased,total_cases,swabs
0,2020-02-24,Abruzzo,0,0,0,0,0,0,0,0,5
1,2020-02-24,Basilicata,0,0,0,0,0,0,0,0,0
2,2020-02-24,Calabria,0,0,0,0,0,0,0,0,1
3,2020-02-24,Campania,0,0,0,0,0,0,0,0,10
4,2020-02-24,Emilia-Romagna,10,2,12,6,18,0,0,18,148
...,...,...,...,...,...,...,...,...,...,...,...
17075,2022-06-26,Toscana,28,1,29,1088,1117,1642,0,2759,11513
17076,2022-06-26,Trentino-Alto Adige,9,0,9,83,92,506,2,600,2699
17077,2022-06-26,Umbria,0,0,0,396,396,559,0,955,3365
17078,2022-06-26,Valle d'Aosta,0,0,0,40,39,5,0,44,256
